In [1]:
import httpx
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.agents.agent_types import AgentType
from crewai.tools import BaseTool
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent, create_sql_agent
import os
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from textwrap import dedent
import litellm
litellm.client_session = httpx.Client(verify=False)

In [2]:
os.environ["OPENAI_API_KEY"]= ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=500, http_client = httpx.Client(verify=False))

#### 1. Hospital Comparison Agent

In [32]:
# Load the CSV file into a Pandas DataFrame
csv_file_path = "../data/Hospital_General_Information.csv"
df = pd.read_csv(csv_file_path)
df.head()

system_message = SystemMessagePromptTemplate.from_template(
"""
You are a highly skilled healthcare assistant with expertise in comparing hospitals. 
Your task is to assess various hospitals based on a user's specific conditions, preferences, and needs. 
You will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, 
and the availability of treatment for specific conditions.

When comparing hospitals, follow these guidelines:

- Condition-Specific Comparison: Focus on the hospitals' expertise in treating the user's specific health condition 
(e.g., heart disease, cancer, etc.).
- Hospital Features: Include details about the hospital's reputation, technology, facilities, specialized care, and any awards or 
recognitions.
- Location and Accessibility: Consider the proximity to the user’s location and the convenience of travel.
- Cost and Insurance: Compare the cost of treatment and insurance coverage options offered by the hospitals.
- Patient Feedback: Analyze reviews and ratings to gauge patient satisfaction and outcomes.
- Personalized Recommendation: Provide a clear, personalized suggestion based on the user’s priorities, whether they are medical 
expertise, convenience, or cost.


Use "Hospital Type" column to look for good facilities of each hospital.
CAREFULLY look at each column name to understand what to output.
"""
)

prompt = ChatPromptTemplate.from_messages([system_message])

pandas_agent = create_pandas_dataframe_agent(llm, df, prompt=prompt, verbose=True, allow_dangerous_code=True, 
                                      agent_type=AgentType.OPENAI_FUNCTIONS,)


class PandasTool(BaseTool):
    name: str = "pandas_tool"
    description: str = "Query pandas dataframe and analyze data"
    
    def _run(self, query: str) -> str:
        return pandas_agent.invoke(query)
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

hospital_info_agent = Agent(
    role='Hospital Information Analyst',
    goal='Analyze hospital data using pandas dataframe and provide relevant output',
    backstory='Expert at analyzing complex datasets using pandas dataframe',
    tools=[PandasTool()],
    verbose=True
)

hospital_info_task = Task(description="which hospital has good medical imaging",
                        expected_output='',
                        agent=hospital_info_agent)

/opt/anaconda3/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'prompt': ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are a highly skilled healthcare assistant with expertise in comparing hospitals. \nYour task is to assess various hospitals based on a user\'s specific conditions, preferences, and needs. \nYou will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, \nand the availability of treatment for specific conditions.\n\nWhen comparing hospitals, follow these guidelines:\n\n- Condition-Specific Comparison: Focus on the hospitals\' expertise in treating the user\'s specific health condition \n(e.g., heart disease, cancer, etc.).\n- Hospital Features: Inclu

#### 2. Doctor Info Agent

In [33]:
tools = SQLDatabaseToolkit(db=SQLDatabase.from_uri("sqlite:///appointments.db"), llm=llm).get_tools()

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Use like operator with lowercase when matching a name. 
When a user is asking to book slots for any dr, STRICTLY Delete the corresponding row from the table.
Begin!

Question: {input}
Thought:{agent_scratchpad}'''

doctor_info_agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=SQLDatabase.from_uri("sqlite:///appointments.db"), llm=llm),
    prompt=PromptTemplate.from_template(template),
    verbose=True,
    handle_parsing_errors=True,
)


class SlotsQueryTool(BaseTool):
    name: str = "sql_query_tool"
    description: str = "Run SQL queries and analyze database data"
    
    def _run(self, query: str) -> str:
        return doctor_info_agent.invoke(query)
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

doctor_slots_agent = Agent(
    role='Doctor Availability Checker and Slot Booking',
    goal='Analyze doctor availability data and books slots if asked',
    backstory='Expert at analyzing complex datasets using SQL',
    tools=[SlotsQueryTool()],
    verbose=True
)

#### 3. Emergency Services Agent

In [34]:
tools = SQLDatabaseToolkit(db=SQLDatabase.from_uri("sqlite:///emergency.db"), llm=llm).get_tools()

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Use like operator with lowercase when matching a name. When user is asking to book slots, delete the corresponding row from the table.
Begin!

Question: {input}
Thought:{agent_scratchpad}'''

emergency_info_agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=SQLDatabase.from_uri("sqlite:///emergency.db"), llm=llm),
    prompt=PromptTemplate.from_template(template),
    verbose=True,
    handle_parsing_errors=True,
)


class EmergencyQueryTool(BaseTool):
    name: str = "sql_query_tool"
    description: str = "Run SQL queries and analyze database data"
    
    def _run(self, query: str) -> str:
        return emergency_info_agent.invoke(query)
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

emergency_agent = Agent(
    role='Emergency Information Finder',
    goal='Analyze emergency services availability data',
    backstory='Expert at analyzing complex datasets using SQL',
    tools=[EmergencyQueryTool()],
    verbose=True
)

#### 4. Diagnostic Tests Agent

In [35]:
diagnostic_agent_system_message = SystemMessagePromptTemplate.from_template(
    """
    You are a highly skilled healthcare assistant with expertise in suggesting health screening tests and packagaes. 
    Your task is to assess various hospitals based on a user's specific conditions, preferences, and needs. 
    You will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, 
    and the availability of treatment for specific conditions.
    
    When comparing hospitals, follow these guidelines:
    
    - Condition-Specific Comparison: Focus on the hospitals' expertise in treating the user's specific health condition 
    (e.g., heart disease, cancer, etc.).
    - Hospital Features: Include details about the hospital's reputation, technology, facilities, specialized care, and any awards or 
    recognitions.
    - Location and Accessibility: Consider the proximity to the user’s location and the convenience of travel.
    - Cost and Insurance: Compare the cost of treatment and insurance coverage options offered by the hospitals.
    - Patient Feedback: Analyze reviews and ratings to gauge patient satisfaction and outcomes.
    - Personalized Recommendation: Provide a clear, personalized suggestion based on the user’s priorities, whether they are medical 
    expertise, convenience, or cost.
    
    CAREFULLY look at each column name to understand what to output.
    """
)

diagnostic_agent = create_pandas_dataframe_agent(llm, pd.read_csv("../data/Hospital_Information_with_Lab_Tests.csv"),
                                                 prompt=ChatPromptTemplate.from_messages([diagnostic_agent_system_message]),
                                                 verbose=True,
                                                 allow_dangerous_code=True,
                                                 agent_type=AgentType.OPENAI_FUNCTIONS, )


class DiagnosticTool(BaseTool):
    name: str = "pandas_tool"
    description: str = "Query pandas dataframe and analyze data"
    
    def _run(self, query: str) -> str:
        return diagnostic_agent.invoke(query)
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not supported")

diagnostic_info_agent = Agent(
    role='Diagnostic Information Finder',
    goal='Analyze diagnostic tests and packages from hospital data',
    backstory='Expert at analyzing complex datasets using csv',
    tools=[DiagnosticTool()],
    verbose=True
)

/opt/anaconda3/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'prompt': ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\n    You are a highly skilled healthcare assistant with expertise in suggesting health screening tests and packagaes. \n    Your task is to assess various hospitals based on a user's specific conditions, preferences, and needs. \n    You will evaluate hospitals considering factors such as medical specialties, patient reviews, location, cost, accessibility, facilities, \n    and the availability of treatment for specific conditions.\n    \n    When comparing hospitals, follow these guidelines:\n    \n    - Condition-Specific Comparison: Focus on the hospitals' expertise in treating the user's specific health condition \n    (e.

In [36]:
hospital_info_task = Task(description="i have an emergency, please provide phone number of the hospital nearby san mateo",
                        expected_output='',
                        agent=hospital_info_agent)

doctor_slots_task = Task(description="show slots for lee",
                         expected_output='',
                         agent=doctor_slots_agent)

emergency_task = Task(description="any ambulance available at 94404",
                      expected_output='',
                      agent=emergency_agent)

diagnostic_info_task = Task(description="what would be preparation instructions for cancer screening",
                        expected_output='',
                        agent=diagnostic_info_agent)

### Sequential Multi Agent Flow

In [9]:
crew = Crew(
    agents=[hospital_info_agent, doctor_slots_agent, emergency_agent, diagnostic_info_agent],
    tasks=[
        hospital_info_task,
        doctor_slots_task,
        emergency_task,
        diagnostic_info_task
    ]
    #process=Process.sequential
)

In [10]:
result = crew.kickoff()

# Agent: Hospital Information Analyst
## Task: i have an emergency, please provide phone number of the hospital nearby san mateo


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Filter the dataframe for hospitals in San Mateo\nsan_mateo_hospitals = df[df['City'].str.upper() == 'SAN MATEO']\n\n# Select the hospital names and phone numbers\nsan_mateo_hospitals_info = san_mateo_hospitals[['Hospital Name', 'Phone Number']]\nsan_mateo_hospitals_info"}`


                 Hospital Name  Phone Number
1185  SAN MATEO MEDICAL CENTER    6505732222The hospital in San Mateo and its phone number is:

- **San Mateo Medical Center**: 650-573-2222

> Finished chain.


# Agent: Hospital Information Analyst
## Thought: I need to provide the phone number of a nearby hospital in San Mateo. To find this, I should analyze the dataset for the required information.
## Using tool: pandas_tool
## Tool Input: 
"{\"query\": \"hospital data for nearby San Mateo and their phon

In [11]:
result

CrewOutput(raw='The query for hospitals offering the "Cancer Screening" health package returned 836 entries. Here are a few examples:\n\n1. **MARSHALL MEDICAL CENTER SOUTH**\n   - Address: 2505 U S HIGHWAY 431 NORTH, BOAZ, AL 35957\n   - Diagnostic Test: MRI Scan\n   - Preparation Instructions: Wear loose and comfortable clothing.\n\n2. **KANAKANAK HOSPITAL**\n   - Address: P O BOX 130, DILLINGHAM, AK 99576\n   - Diagnostic Test: X-Ray\n   - Preparation Instructions: Avoid caffeine for 24 hours.\n\n3. **TUBA CITY REGIONAL HEALTH CARE CORPORATION**\n   - Address: PO BOX 600, TUBA CITY, AZ 86045\n   - Diagnostic Test: Cholesterol Test\n   - Preparation Instructions: Fast for 12 hours before the test.\n\n4. **SUTTER ROSEVILLE MEDICAL CENTER**\n   - Address: ONE MEDICAL PLAZA, ROSEVILLE, CA 95661\n   - Diagnostic Test: Blood Test\n   - Preparation Instructions: Drink plenty of water.\n\n5. **ST FRANCIS HOSPITAL**\n   - Address: 7TH AND CLAYTON STS, WILMINGTON, DE 19805\n   - Diagnostic Tes

In [12]:
for task in result.tasks_output:
    print(task.raw)
    print("\n\n")

The hospital in San Mateo and its phone number is:

- **San Mateo Medical Center**: 650-573-2222



Dr. Ronny Lee has available slots on 2024-12-22 at 1:00 PM and 3:00 PM.



There are no available ambulances in the zip code 94404 according to the database.



The query for hospitals offering the "Cancer Screening" health package returned 836 entries. Here are a few examples:

1. **MARSHALL MEDICAL CENTER SOUTH**
   - Address: 2505 U S HIGHWAY 431 NORTH, BOAZ, AL 35957
   - Diagnostic Test: MRI Scan
   - Preparation Instructions: Wear loose and comfortable clothing.

2. **KANAKANAK HOSPITAL**
   - Address: P O BOX 130, DILLINGHAM, AK 99576
   - Diagnostic Test: X-Ray
   - Preparation Instructions: Avoid caffeine for 24 hours.

3. **TUBA CITY REGIONAL HEALTH CARE CORPORATION**
   - Address: PO BOX 600, TUBA CITY, AZ 86045
   - Diagnostic Test: Cholesterol Test
   - Preparation Instructions: Fast for 12 hours before the test.

4. **SUTTER ROSEVILLE MEDICAL CENTER**
   - Address: ONE MEDI

### Multi Agent Flow with router

In [37]:
router_agent = Agent(
    role='Researcher',
    goal='Research and analyze information',
    backstory='Expert at gathering and analyzing information',
    allow_delegation=True
)

In [38]:
query = "book 1 PM slot for dr lee"

routing_task = Task(
        description=dedent(f"""
            Route this query: "{query}"
            Choose the most appropriate agent based on keywords and context.
            Return the output of the selected agent.
        """),
        expected_output='',
        agent=router_agent
)

multi_agent_crew = Crew(
    agents=[router_agent, hospital_info_agent, doctor_slots_agent, emergency_agent, diagnostic_info_agent],
    tasks=[routing_task]
)

Overriding of current TracerProvider is not allowed


In [39]:
result_with_router = multi_agent_crew.kickoff()

# Agent: Doctor Availability Checker and Slot Boooking
## Task: Book a 1 PM slot for Dr. Lee.


> Entering new SQL Agent Executor chain...
The question is asking for the availability of Dr. Lee at a specific time slot. To answer this, I need to check if the table `doctor_availability` exists and then verify the schema to ensure the columns `doctor_name` and `time_slot` are present. 

Action: sql_db_list_tables
Action Input: doctors, slotsIt seems that the table `doctor_availability` does not exist. Instead, there are tables named `doctors` and `slots`. I should check the schema of these tables to determine if they contain the necessary information about doctor availability.

Action: sql_db_schema
Action Input: doctors, slots
CREATE TABLE doctors (
	id INTEGER, 
	name TEXT NOT NULL, 
	specialization TEXT NOT NULL, 
	contact TEXT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from doctors table:
id	name	specialization	contact
1	Michael Smith	Cardiology	1234567890
2	Ronny Lee	Dermatology	98765

In [40]:
for task in result_with_router.tasks_output:
    print(task.raw)
    print("\n\n")

The 1 PM slot for Dr. Lee on December 22, 2024, has been successfully booked for you.



